In [136]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

np.random.seed(123)

In [137]:
# NETWORK = 'InceptionResNetV2'
# NETWORK = 'InceptionV3'
# NETWORK = 'Resnet'
# NETWORK = 'VGG16'
# NETWORK = 'VGG19'
# NETWORK = 'Xception'
# NETWORK = 'DenseNet201'
# NETWORK = 'EfficientNetB1'
# NETWORK = 'EfficientNetB4'
NETWORK = 'EfficientNetB7'
# NETWORK = 'NASNetLarge'
# NETWORK = 'ConvNeXtBase'

METHOD = "MED-STD"

In [138]:
WORKING_DIRECTORY = '/datassd/WHOLEIMAGE_MAMIP'
PET_features_filename = f"Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"

features_path = os.path.join(WORKING_DIRECTORY, f"EXTRACTED_FEATURES/ExtractedFeatures_{METHOD}")
processed_features_path = os.path.join(WORKING_DIRECTORY, f"PROCESSED_FEATURES/ProcessedFeatures_{METHOD}")
if not os.path.exists(processed_features_path):
    os.makedirs(processed_features_path)


processed_features_filename = f"Processed_Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"
processed_features_full_filename = os.path.join(processed_features_path, processed_features_filename)

PET_featurs_full_path = os.path.join(features_path, PET_features_filename)

PET_filename = PET_features_filename.split(".")[0]

PET_outcome_data = pd.read_csv(PET_featurs_full_path)

outcome_file = "/datassd/WHOLEIMAGE_MAMIP/hecktor2022_patient_endpoint_training.csv"

In [139]:
PET_outcome_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,CHUM-001,0.070712,0.045909,0.074574,0.079520,0.172740,0.065550,0.084957,0.012624,0.082553,...,0.014804,0.092372,0.035536,0.019154,0.051116,0.058740,0.063747,0.052868,0.036274,0.043196
1,CHUM-002,0.089442,0.037251,0.020300,0.088806,0.094708,0.093244,0.108037,0.022975,0.084200,...,0.017853,0.026420,0.029107,0.061149,0.016072,0.008838,0.094208,0.024927,0.013076,0.112583
2,CHUM-006,0.081624,0.055480,0.079067,0.033811,0.119887,0.172648,0.145843,0.017671,0.128864,...,0.013282,0.185855,0.024869,0.019575,0.131839,0.050627,0.236354,0.033627,0.064434,0.125622
3,CHUM-007,0.086418,0.071077,0.082775,0.069877,0.059129,0.090658,0.099781,0.014217,0.129869,...,0.009232,0.092623,0.012301,0.019842,0.026175,0.089430,0.337084,0.023467,0.035467,0.078264
4,CHUM-008,0.073979,0.012672,0.041864,0.076115,0.109919,0.112156,0.156645,0.021195,0.194249,...,0.013573,0.282091,0.128687,0.014792,0.038197,0.123330,0.110062,0.047179,0.045671,0.050455


# Up to here

Read Outcome csv

In [140]:
OD = pd.read_csv(outcome_file)
OD = OD.iloc[:,:]
OD.head()

,PatientID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [141]:
OD = OD.rename(columns={'PatientID': 'Patient_ID'})
OD.head()

,Patient_ID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [142]:
OD.columns[0]

'Patient_ID'

In [143]:
first_column = PET_outcome_data.columns[0]
PET_outcome_data = PET_outcome_data.rename(columns={first_column: 'Patient_ID'})

In [144]:
OD.columns[0]


'Patient_ID'

In [145]:
PET_outcome_data.columns[0]

'Patient_ID'

In [146]:
PET_outcome_data.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,CHUM-001,0.070712,0.045909,0.074574,0.079520,0.172740,0.065550,0.084957,0.012624,0.082553,...,0.014804,0.092372,0.035536,0.019154,0.051116,0.058740,0.063747,0.052868,0.036274,0.043196
1,CHUM-002,0.089442,0.037251,0.020300,0.088806,0.094708,0.093244,0.108037,0.022975,0.084200,...,0.017853,0.026420,0.029107,0.061149,0.016072,0.008838,0.094208,0.024927,0.013076,0.112583
2,CHUM-006,0.081624,0.055480,0.079067,0.033811,0.119887,0.172648,0.145843,0.017671,0.128864,...,0.013282,0.185855,0.024869,0.019575,0.131839,0.050627,0.236354,0.033627,0.064434,0.125622
3,CHUM-007,0.086418,0.071077,0.082775,0.069877,0.059129,0.090658,0.099781,0.014217,0.129869,...,0.009232,0.092623,0.012301,0.019842,0.026175,0.089430,0.337084,0.023467,0.035467,0.078264
4,CHUM-008,0.073979,0.012672,0.041864,0.076115,0.109919,0.112156,0.156645,0.021195,0.194249,...,0.013573,0.282091,0.128687,0.014792,0.038197,0.123330,0.110062,0.047179,0.045671,0.050455


In [147]:
# ranges_list = list()
# for col in PET_outcome_data.iloc[:, 1:]:
#     col_range = PET_outcome_data[col].max() - PET_outcome_data[col].min()
#     ranges_list.append(col_range)
#     print(f"Range of column {col}: {col_range}")
# print(f"maximum range: {max(ranges_list)}")

In [148]:
full_features_df = pd.merge(PET_outcome_data, OD, on=OD.columns[0])

In [149]:
full_features_df.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,2552,2553,2554,2555,2556,2557,2558,2559,Relapse,RFS
0,CHUM-001,0.070712,0.045909,0.074574,0.079520,0.172740,0.065550,0.084957,0.012624,0.082553,...,0.035536,0.019154,0.051116,0.058740,0.063747,0.052868,0.036274,0.043196,0,1704
1,CHUM-002,0.089442,0.037251,0.020300,0.088806,0.094708,0.093244,0.108037,0.022975,0.084200,...,0.029107,0.061149,0.016072,0.008838,0.094208,0.024927,0.013076,0.112583,1,439
2,CHUM-006,0.081624,0.055480,0.079067,0.033811,0.119887,0.172648,0.145843,0.017671,0.128864,...,0.024869,0.019575,0.131839,0.050627,0.236354,0.033627,0.064434,0.125622,0,1186
3,CHUM-007,0.086418,0.071077,0.082775,0.069877,0.059129,0.090658,0.099781,0.014217,0.129869,...,0.012301,0.019842,0.026175,0.089430,0.337084,0.023467,0.035467,0.078264,0,1702
4,CHUM-008,0.073979,0.012672,0.041864,0.076115,0.109919,0.112156,0.156645,0.021195,0.194249,...,0.128687,0.014792,0.038197,0.123330,0.110062,0.047179,0.045671,0.050455,0,1499


In [150]:
full_features_df.to_csv(processed_features_full_filename, encoding='utf-8', index=False)